In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np

In [3]:
df=pd.read_csv('train.csv')

In [4]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
df=df.dropna()

In [6]:
X=df.drop('label',axis=1)

In [7]:
y=df['label']

In [8]:
X.shape

(18285, 4)

In [9]:
y.shape

(18285,)

In [10]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [11]:
###Vocabulary size
voc_size=5000

In [12]:
messages=X.copy()

In [13]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [14]:
messages.reset_index(inplace=True)

In [15]:
import nltk
import re
from nltk.corpus import stopwords

In [16]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [17]:
###Dataset preprocessing
from nltk.stem.porter import PorterStemmer
ps= PorterStemmer()
corpus= []
for i in range(0,len(messages)):
  review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [18]:
corpus[:5]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri']

In [21]:
len(corpus)

18285

In [22]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]

In [23]:
onehot_repr[:5]

[[413, 1396, 4107, 1084, 2758, 3474, 3114, 3016, 3853, 1063],
 [4070, 640, 1881, 3954, 4728, 797, 1733],
 [3441, 2762, 584, 712],
 [4785, 3515, 3292, 146, 4087, 3957],
 [840, 4728, 4826, 1537, 487, 3119, 4728, 1335, 115, 4101]]

In [24]:
###every sentence have different length so to make it same length, use pad_sequence
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
embedded_docs[:5]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  413,
        1396, 4107, 1084, 2758, 3474, 3114, 3016, 3853, 1063],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0, 4070,  640, 1881, 3954, 4728,  797, 1733],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0, 3441, 2762,  584,  712],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0, 4785, 3515, 3292,  146, 4087, 3957],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  840,
        4728, 4826, 1537,  487, 3119, 4728, 1335,  115, 4101]],
      dtype=int32)

In [25]:
##Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [26]:
len(embedded_docs),y.shape

(18285, (18285,))

In [27]:
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [28]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [30]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 9s 35ms/step - loss: 0.4701 - accuracy: 0.7517 - val_loss: 0.1963 - val_accuracy: 0.9185
Epoch 2/10
192/192 [==============================] - 6s 33ms/step - loss: 0.1338 - accuracy: 0.9493 - val_loss: 0.1875 - val_accuracy: 0.9221
Epoch 3/10
192/192 [==============================] - 7s 35ms/step - loss: 0.0887 - accuracy: 0.9686 - val_loss: 0.2076 - val_accuracy: 0.9186
Epoch 4/10
192/192 [==============================] - 7s 34ms/step - loss: 0.0688 - accuracy: 0.9738 - val_loss: 0.2354 - val_accuracy: 0.9152
Epoch 5/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0474 - accuracy: 0.9843 - val_loss: 0.2816 - val_accuracy: 0.9056
Epoch 6/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0419 - accuracy: 0.9875 - val_loss: 0.2957 - val_accuracy: 0.9148
Epoch 7/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0332 - accuracy: 0.9893 - val_loss: 0.3596 - val_accuracy: 0.9094

In [31]:
y_pred=model.predict_classes(X_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [32]:
from sklearn.metrics import confusion_matrix

In [33]:
confusion_matrix(y_test,y_pred)

array([[3121,  298],
       [ 215, 2401]])

In [34]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9149958574979288

Adding Dropout

In [ ]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])